# Selección del clasificador

Alfredo Rodríguez Magdalena ©

In [1]:
import warnings
warnings.filterwarnings("ignore")

import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

## Importación de los datos de entrenamiento

In [2]:
data = mat73.loadmat(r'../datos/experimento3_sep.mat')
labels = scipy.io.loadmat(r'../datos/labelsExperimento3.mat')
Vr14 = scipy.io.loadmat(r'../datos/Vr14.mat')

X = data["X"]
U = data["U"]
N = data["N"]
Y = labels['Y']
Vr14 = Vr14['Vr14']

tm = 5e-3 # Tiempo de muestreo

print(f"Dimensiones de las etiquetas : {Y.shape}")

Dimensiones de las etiquetas : (11953, 1)


In [3]:
scalerX = StandardScaler()
Xnormalizado = scalerX.fit_transform(X)
Unormalizado = U[:,0] / 1000 # Se expresa en m/s

In [4]:
# DS1 -> Todos los datos sin tratar
X1 = np.hstack((Xnormalizado, Unormalizado.reshape(-1,1)))

print(f"Dimensiones de dataset 1 : {X1.T.shape} (características, muestras)")

Dimensiones de dataset 1 : (501, 11953) (características, muestras)


In [5]:
# DS2 -> Datos en la base de Vr14
X2 = Vr14 @ X.T
X2 = np.hstack((X2.T, U[:,0].reshape(-1,1)))

print(f"Dimensiones de la matriz de cambio de base : {Vr14.shape}")
print(f"Dimensiones de dataset 2 : {X2.T.shape} (características, muestras)")

Dimensiones de la matriz de cambio de base : (15, 500)
Dimensiones de dataset 2 : (16, 11953) (características, muestras)


In [6]:
# DS3 -> Datos deducidos a partir de la etapa de extracción de características superficial
X3 = data["F"]
X3 = np.hstack((X3, U[:,0].reshape(-1,1)))

print(f"Dimensiones de dataset 3 : {X3.T.shape} (características, muestras)")

Dimensiones de dataset 3 : (3, 11953) (características, muestras)


In [7]:
datasets = {"DS1" : X1, 
            "DS2" : X2, 
            "DS3" : X3}

## Entrenamiento de modelos

In [8]:
semilla = 92
np.random.seed(semilla) # Semilla para reproducibilidad

In [9]:
# Diccionario con los algoritmos de clasificación
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Gaussian Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "SGD Classifier": SGDClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "ANN": MLPClassifier(max_iter=300),
}

In [10]:
print(np.ravel(Y).shape)

(11953,)


In [41]:
# Función para entrenar y evaluar cada algoritmo en un dataset
results = []
decision_trees = []

for dataset_name, XX in datasets.items():
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(XX, np.ravel(Y), test_size=0.3, random_state=semilla)

    for model_name, model in classifiers.items():
        # Entrenar el modelo
        model.fit(X_train, y_train)
        
        # Medir tiempo de predicción
        start_time = time.time()
        y_pred = model.predict(X_test)
        prediction_time = time.time() - start_time  # Tiempo en segundos

        if model_name == "Decision Tree" and dataset_name == "DS2":
            print(X_test.shape)
            
            tree_text = export_text(model, max_depth=model.tree_.max_depth)
            print(tree_text)

        # Calcular las métricas
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        results.append({
            "Dataset": dataset_name,
            "Model": model_name,
            "Precision": report["weighted avg"]["precision"],
            "Recall": report["weighted avg"]["recall"],
            "F1-Score": report["weighted avg"]["f1-score"],
            "Prediction Time [ms]": prediction_time * 10 # Más tarde se multiplica por 100 para obtener ms
        })

# Convertir resultados a un DataFrame
results_df = pd.DataFrame(results)


(3586, 16)
|--- feature_15 <= 750.00
|   |--- feature_2 <= -2.69
|   |   |--- feature_0 <= -47.79
|   |   |   |--- class: 2.0
|   |   |--- feature_0 >  -47.79
|   |   |   |--- feature_12 <= -0.22
|   |   |   |   |--- feature_14 <= -0.43
|   |   |   |   |   |--- class: 2.0
|   |   |   |   |--- feature_14 >  -0.43
|   |   |   |   |   |--- class: 0.0
|   |   |   |--- feature_12 >  -0.22
|   |   |   |   |--- class: 0.0
|   |--- feature_2 >  -2.69
|   |   |--- feature_2 <= 0.53
|   |   |   |--- feature_4 <= 1.35
|   |   |   |   |--- feature_8 <= 2.12
|   |   |   |   |   |--- feature_0 <= -35.05
|   |   |   |   |   |   |--- feature_12 <= 2.49
|   |   |   |   |   |   |   |--- feature_8 <= 1.57
|   |   |   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |   |   |--- feature_8 >  1.57
|   |   |   |   |   |   |   |   |--- feature_5 <= -0.31
|   |   |   |   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |   |   |   |--- feature_5 >  -0.31
|   |   |   |   |   |   |   |   |   |--- 

In [34]:
numeric_columns = results_df.select_dtypes(include=[np.number]).columns
results_df[numeric_columns] = results_df[numeric_columns].map(lambda x: f"{x*100:.2f}")

results_df.to_csv("resultadosClasificacion.csv", index=False)

In [39]:
print(f"El tamaño del vector que necesita como entrada el árbol de decisión es: {decision_trees[1].n_features_in_}")

El tamaño del vector que necesita como entrada el árbol de decisión es: 3


In [36]:
# Exportar el árbol de decisión a texto sin truncar las ramas
tree_text = export_text(decision_trees[1], max_depth=decision_trees[1].tree_.max_depth)
print(tree_text)

|--- feature_2 <= 750.00
|   |--- feature_0 <= 5.29
|   |   |--- feature_0 <= 3.95
|   |   |   |--- feature_1 <= 11.15
|   |   |   |   |--- feature_1 <= 10.78
|   |   |   |   |   |--- feature_0 <= 0.72
|   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |--- feature_0 >  0.72
|   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |--- feature_1 >  10.78
|   |   |   |   |   |--- feature_1 <= 10.78
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |--- feature_1 >  10.78
|   |   |   |   |   |   |--- feature_0 <= 3.90
|   |   |   |   |   |   |   |--- feature_0 <= 2.95
|   |   |   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |   |   |--- feature_0 >  2.95
|   |   |   |   |   |   |   |   |--- feature_0 <= 2.95
|   |   |   |   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |   |   |   |--- feature_0 >  2.95
|   |   |   |   |   |   |   |   |   |--- feature_0 <= 3.09
|   |   |   |   |   |   |   |   |   |   |--- feature_0 <= 3.08
|   |   |   |   |   |   | 